### Logistic function

Here's another straightforward example, though a bit more elaborate than adding two numbers together.  Let's say that you want to compute the logistic curve, which is given by:

$$s(x) = \frac{1}{1 + e^{-x}} $$

In [6]:
import theano.tensor as T
from theano import function
x = T.dmatrix('x')
s = 1 / (1 + T.exp(-x))
logistic = function([x],s)
logistic([[0,1],[-1,-2]])

array([[ 0.5       ,  0.73105858],
       [ 0.26894142,  0.11920292]])

We can alternatively compute:

$$s(x) = \frac{1}{1+e^{-x}} = \frac{1+\text{tanh}(x/2)}{2} $$

In [8]:
s2 = (1 + T.tanh(x/2)) / 2
logistic2 = function([x],s2)
logistic2([[0,1],[-1,-2]])

array([[ 0.5       ,  0.73105858],
       [ 0.26894142,  0.11920292]])

### Computing more than one thing at the same time

In [11]:
a, b = T.dmatrices('a','b')
diff = a - b
abs_diff = abs(diff)
diff_squared = diff**2
f = function([a,b], [diff, abs_diff, diff_squared])
f([[1, 1], [1, 1]], [[0, 1], [2, 3]])

[array([[ 1.,  0.],
        [-1., -2.]]), array([[ 1.,  0.],
        [ 1.,  2.]]), array([[ 1.,  0.],
        [ 1.,  4.]])]

### Setting a default value for an argument

In [17]:
from theano import Param
x,y = T.dscalars('x','y')
z = x + y
f = function([x, Param(y,default=1)], z)
print f(33)
print f(33, 2)

34.0
35.0


In [19]:
x, y, w = T.dscalars('x', 'y', 'w')
z = (x+y)*w
f = function([x, Param(y,default=1), Param(w, default=2, name='w_by_name')], z)
print f(33)
print f(33,2)
print f(33,0,1)
print f(33, w_by_name=1)
print f(33, w_by_name=1, y = 0)

68.0
70.0
33.0
34.0
33.0


### Using shared variables

In [22]:
from theano import shared
state = shared(0)
inc = T.iscalar('inc')
accumulator = function([inc], state, updates=[(state, state+inc)])
print state.get_value()
print accumulator(1)
print state.get_value()
print accumulator(300)
print state.get_value()


0
0
1
1
301


In [24]:
state.set_value(-1)
print accumulator(3)
print state.get_value()

-1
2


In [26]:
decrementor = function([inc], state, updates = [(state, state-inc)])
decrementor(10)
print state.get_value()

-8


### Using random numbers

Because in Theano you first express everything symbolically and afterwards compile this expression to get functions, using psuedo-random numbers is not as straightforward as it is in NumPy, though also not too complicated.

...

### A real example: logistic regression

In [29]:
import numpy
import theano
import theano.tensor as T
rng = numpy.random

N = 400
feats = 784
D = (rng.randn(N,feats), rng.randint(size=N, low=0, high=2))
training_steps = 10000

# Declare Theano symbolic variables
x = T.matrix("x")
y = T.vector("y")
w = theano.shared(rng.randn(feats), name="w")
b = theano.shared(0., name="b")
print "Initial model:"
print w.get_value(), b.get_value()

# Construct Theano expression graph
p_1 = 1 / (1 + T.exp(-T.dot(x,w) - b))
prediction = p_1 > 0,5
xent = -y * T.log(p_1) - (1-y) * T.log(1-p_1)
cost = xent.mean() + 0.01 * (w ** 2).sum()
gw, gb = T.grad(cost, [w,b])




Initial model:
[ -1.09100692e+00   9.96886551e-01   6.47216403e-01  -1.42549013e+00
  -7.00632166e-02   4.27372143e-01  -3.74751576e-01  -3.24777185e-01
   3.80988698e-01  -9.15217193e-01  -4.24786005e-01   1.13010058e+00
  -4.78531438e-01   1.97691897e-01  -1.08203078e+00   6.33126756e-01
   6.52064524e-01   4.19761486e-01  -1.84573115e+00  -1.06525396e-01
   5.63055941e-02  -1.30977981e+00  -8.47918885e-01  -6.44049592e-01
   6.63900110e-01  -5.05105434e-01  -1.38112003e-01   6.97506427e-02
   6.48460395e-01   1.14079493e+00   5.90217335e-01  -6.00217215e-01
  -9.48464111e-01   2.38122135e+00  -4.84045421e-01  -6.25002320e-01
   2.78281079e-01   5.26260551e-01  -4.86973267e-01   6.87439911e-01
   1.24500789e-02   1.26332186e-01   7.17591905e-02   8.25590162e-01
   9.42859938e-01   1.60028017e+00   2.38427540e-01   1.49742280e-01
  -1.70039211e+00  -7.27362389e-01   1.41403880e+00  -2.48772163e+00
   5.64400608e-01   4.04273705e-01   1.35599573e+00  -1.03984617e-01
   4.82729971e-02  

In [31]:
train = theano.function(
          inputs=[x,y],
          outputs=[prediction, xent],
          updates=((w, w - 0.1 * gw), (b, b - 0.1 * gb)))
predict = theano.function(inputs=[x], outputs=prediction)

# Train
for i in range(training_steps):
    pred, err = train(D[0], D[1])
    
print "Final model:"
print w.get_value(), b.get_value()
print "target values for D:", D[1]
print "prediction on D:", predict(D[0])

TypeError: Outputs must be theano Variable or Out instances. Received (Elemwise{gt,no_inplace}.0, 5) of type <type 'tuple'>